In [1]:
# import VAE from other script
import os
from MicrobeVAE import VariationalAutoencoder

# import necessary packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# disable eager execution
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [2]:
# load fracking data and make necessary changes to shape
combined_metadata = pd.read_csv(r"C:\Users\Peter\Desktop\Classes\Summer 2021\CSI Research\edited_mt_combined_metadata.txt", sep = "\t")
species_table = pd.read_csv(r"C:\Users\Peter\Desktop\Classes\Summer 2021\CSI Research\norm_filtered_species_table.txt", sep = "\t")

# set metadata index to sample ID
combined_metadata = combined_metadata.set_index("SampleID")

# set species table index to Sample ID
species_table = species_table.set_index("SampleID")

# transpose species table
species_table = species_table.transpose()

# merge metadata and species data
merged_data = pd.merge(species_table, combined_metadata, left_index=True, right_index=True)
merged_data.head()

,d__Archaea;p__Candidatus_Korarchaeota;g__Candidatus_Korarchaeum;s__Candidatus_Korarchaeum_cryptofilum,d__Archaea;p__Candidatus_Lokiarchaeota;g__Candidatus_Prometheoarchaeum;s__Candidatus_Prometheoarchaeum_syntrophicum,d__Archaea;p__Candidatus_Micrarchaeota;g__Candidatus_Mancarchaeum;s__Candidatus_Mancarchaeum_acidiphilum,d__Archaea;p__Candidatus_Micrarchaeota;g__Candidatus_Micrarchaeum;s__Candidatus_Micrarchaeum_sp.,d__Archaea;p__Candidatus_Thermoplasmatota;c__Thermoplasmata;o__Methanomassiliicoccales;f__Candidatus_Methanomethylophilaceae;g__Candidatus_Methanomethylophilus;s__Candidatus_Methanomethylophilus_alvus,d__Archaea;p__Candidatus_Thermoplasmatota;c__Thermoplasmata;o__Methanomassiliicoccales;f__Methanomassiliicoccaceae;g__Candidatus_Methanoplasma;s__Candidatus_Methanoplasma_termitum,d__Archaea;p__Candidatus_Thermoplasmatota;c__Thermoplasmata;o__Methanomassiliicoccales;f__Methanomassiliicoccaceae;g__Methanomassiliicoccus;s__Candidatus_Methanomassiliicoccus_intestinalis,d__Archaea;p__Candidatus_Thermoplasmatota;c__Thermoplasmata;o__Thermoplasmatales;f__Cuniculiplasmataceae;g__Cuniculiplasma;s__Cuniculiplasma_divulgatum,d__Archaea;p__Candidatus_Thermoplasmatota;c__Thermoplasmata;o__Thermoplasmatales;f__Ferroplasmaceae;g__Ferroplasma;s__Ferroplasma_acidarmanus,d__Archaea;p__Candidatus_Thermoplasmatota;c__Thermoplasmata;o__Thermoplasmatales;f__Ferroplasmaceae;g__Ferroplasma;s__Ferroplasma_acidiphilum,...,Lat,Long,Date,HF_Status,Year,R1File,Matrix,HF_Status_Matrix,Year_Matrix,name
AB1_S65_L006,0.850561,109.297,21.2640,28.06850,155.2270,57.4129,253.4670,56.13710,0.0,0.0,...,NaN,NaN,NaN,HF+,Y2019,/home/see/Wright_Labs/May_2021/Fracking_data_s...,Sediment,Sediment_HF+,Sediment_Y2019,Alex.Branch.Sediment.1
AB3_S66_L006,2.266840,121.049,35.3627,23.12170,178.6270,63.9248,301.4890,68.00510,0.0,0.0,...,NaN,NaN,NaN,HF+,Y2019,/home/see/Wright_Labs/May_2021/Fracking_data_s...,Sediment,Sediment_HF+,Sediment_Y2019,Alex.Branch.Sediment.3
BCD1_S67_L006,2.969830,223.727,22.7687,1.97989,116.8130,22.2738,85.1352,7.42458,0.0,0.0,...,NaN,NaN,NaN,HF+,Y2019,/home/see/Wright_Labs/May_2021/Fracking_data_s...,Sediment,Sediment_HF+,Sediment_Y2019,Black.Creek.Downstream.Sediment.1
BCD2_S68_L006,2.212400,110.620,57.9649,12.83190,135.8410,146.4610,336.2850,39.38070,0.0,0.0,...,NaN,NaN,NaN,HF+,Y2019,/home/see/Wright_Labs/May_2021/Fracking_data_s...,Sediment,Sediment_HF+,Sediment_Y2019,Black.Creek.Downstream.Sediment.2
BCU1_S69_L006,7.245690,227.204,17.0791,13.45630,64.6937,63.1410,534.6290,86.43080,0.0,0.0,...,NaN,NaN,NaN,HF-,Y2019,/home/see/Wright_Labs/May_2021/Fracking_data_s...,Sediment,Sediment_HF-,Sediment_Y2019,Black.Creek.Upstream.Sediment.1


In [3]:
# define X (input): remove metadata tags
X = merged_data.iloc[:, :-17]

# define y (output): Select only HF status
y = merged_data.loc[:, "HF_Status"]
y = np.where(y == "HF+", 1, 0)

# convert y to float32 so true values and preds are same data type, then convert to keras tensor
y = y.astype('float32')

print(X.shape, y.shape)

(99, 8218) (99,)


In [4]:
# split X and y into train and test sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(79, 8218) (20, 8218) (79,) (20,)


In [5]:
# create the VAE
vae = VariationalAutoencoder(
    input_dim = (X_train.shape[1],),
    encoder_layer_size = [2048, 1024, 512],
    decoder_layer_size = [512, 1024, 2048, X_train.shape[1]],
    z_dim = 256)

In [6]:
# encoder summary
vae.encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 8218)]       0                                            
__________________________________________________________________________________________________
encoder_dense_0 (Dense)         (None, 2048)         16832512    encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 2048)         0           encoder_dense_0[0][0]            
__________________________________________________________________________________________________
encoder_dense_1 (Dense)         (None, 1024)         2098176     leaky_re_lu[0][0]                
____________________________________________________________________________________________

In [7]:
# decoder summary
vae.decoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 256)]             0         
_________________________________________________________________
decoder_dense_0 (Dense)      (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
decoder_dense_1 (Dense)      (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
decoder_dense_2 (Dense)      (None, 2048)              2099200   
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 2048)              0   

In [8]:
## TRAIN and COMPILE the VAE
lr = 0.0005
r_loss_factor = 10000
epochs = 1000
batch_size = 16

In [9]:
vae.compile(learning_rate = lr, r_loss_factor = r_loss_factor)

In [10]:
vae.train(x_train = X_train, y_train = X_train,
         epochs = epochs,
         batch_size = batch_size)

Train on 66 samples, validate on 13 samples
Epoch 1/1000
66/66 [==============================] - 2s 25ms/sample - loss: nan - vae_r_loss: 10659589120.0000 - vae_kl_loss: nan - val_loss: nan - val_vae_r_loss: 13956357120.0000 - val_vae_kl_loss: nan
Epoch 2/1000
48/66 [====================>.........] - ETA: 0s - loss: nan - vae_r_loss: 11286819840.0000 - vae_kl_loss: nan

C:\Users\Peter\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


66/66 [==============================] - 0s 3ms/sample - loss: nan - vae_r_loss: 11506907136.0000 - vae_kl_loss: nan - val_loss: nan - val_vae_r_loss: 13956357120.0000 - val_vae_kl_loss: nan
Epoch 3/1000
66/66 [==============================] - 0s 3ms/sample - loss: nan - vae_r_loss: 10958352384.0000 - vae_kl_loss: nan - val_loss: nan - val_vae_r_loss: 13956357120.0000 - val_vae_kl_loss: nan
Epoch 4/1000
66/66 [==============================] - 0s 3ms/sample - loss: nan - vae_r_loss: 11157277696.0000 - vae_kl_loss: nan - val_loss: nan - val_vae_r_loss: 13956357120.0000 - val_vae_kl_loss: nan
Epoch 5/1000
66/66 [==============================] - 0s 3ms/sample - loss: nan - vae_r_loss: 10679817216.0000 - vae_kl_loss: nan - val_loss: nan - val_vae_r_loss: 13956357120.0000 - val_vae_kl_loss: nan
Epoch 6/1000
66/66 [==============================] - 0s 3ms/sample - loss: nan - vae_r_loss: 10592413696.0000 - vae_kl_loss: nan - val_loss: nan - val_vae_r_loss: 13956357120.0000 - val_vae_kl_los

KeyboardInterrupt: 